## Check for CUDA

In [1]:
import torch
print(torch.cuda.is_available())

False


/home/melihkaan/Desktop/vision/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


## Import AID Dataset

In [18]:
import os
import torch
from torchvision import datasets, transforms

data_dir = "datasets/AID"
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)

# Decide on the ratio of train/val/test
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size]
)


print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print(f"Train dataset: {train_dataset[0]}")

Train dataset size: 6993
Val dataset size: 1498
Test dataset size: 1500
Train dataset: (tensor([[[0.2235, 0.2235, 0.2196,  ..., 0.4392, 0.4039, 0.8471],
         [0.2235, 0.2275, 0.2196,  ..., 0.4431, 0.3686, 0.6941],
         [0.2196, 0.2235, 0.2157,  ..., 0.4588, 0.4118, 0.4980],
         ...,
         [0.4784, 0.5098, 0.5255,  ..., 0.2549, 0.2549, 0.2627],
         [0.5176, 0.4824, 0.4863,  ..., 0.2314, 0.2353, 0.1961],
         [0.5176, 0.5020, 0.4588,  ..., 0.2157, 0.2510, 0.2784]],

        [[0.2235, 0.2235, 0.2157,  ..., 0.3294, 0.3059, 0.8275],
         [0.2157, 0.2118, 0.2000,  ..., 0.3216, 0.2627, 0.6745],
         [0.2157, 0.2157, 0.2039,  ..., 0.3137, 0.2902, 0.4588],
         ...,
         [0.5255, 0.5529, 0.5608,  ..., 0.2353, 0.2392, 0.2471],
         [0.5569, 0.5294, 0.5255,  ..., 0.2157, 0.2157, 0.1725],
         [0.5569, 0.5451, 0.4980,  ..., 0.2000, 0.2275, 0.2549]],

        [[0.2235, 0.2235, 0.2235,  ..., 0.3137, 0.2902, 0.8000],
         [0.2314, 0.2235, 0.2196,  

## Augment and Normalize the Images

In [19]:
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    # Common augmentations for aerial imagery
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [20]:
num_classes = len(os.listdir(data_dir))

In [21]:
batch_size = 16

train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_test_transform
test_dataset.dataset.transform = val_test_transform

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [22]:
import torch
import torch.nn as nn
import torchvision.models as models

model = models.densenet121(pretrained=True)
model.classifier = nn.Linear(model.classifier.in_features, len(train_dataset.dataset.classes))
criterion = nn.CrossEntropyLoss()

In [23]:
import torch

# Free up all unused memory
torch.cuda.empty_cache()


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)


In [27]:
import optuna
def objective(trial):
    # Suggest hyperparameters
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD"])
    momentum = trial.suggest_float("momentum", 0.5, 0.99)

    # Create model again for each trial
    model_ = models.densenet121(pretrained=True)
    model_.classifier = nn.Linear(model.classifier.in_features, len(train_dataset.dataset.classes))
    model_.to(device)

    # Choose an optimizer based on the suggestion
    if optimizer_name == "Adam":
        opt = torch.optim.Adam(model_.parameters(), lr=lr)
    else:  # SGD
        opt = torch.optim.SGD(model_.parameters(), lr=lr, momentum=momentum)

    # Training loop (small number of epochs for demonstration)
    max_epochs = 3
    for epoch in range(max_epochs):
        model_.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            opt.zero_grad()
            outputs = model_(images)
            loss = criterion(outputs, labels)
            loss.backward()
            opt.step()

        # Validation to measure accuracy
        model_.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model_(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        
        # Report validation accuracy to optuna
        trial.report(val_accuracy, epoch)

        # Handle pruning (optional)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return val_accuracy

# Run optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)


[I 2025-02-11 21:56:37,333] A new study created in memory with name: no-name-690b8f28-e62c-4765-acac-8edb27a15ec6
/tmp/ipykernel_23483/2115594751.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
[I 2025-02-11 22:00:18,699] Trial 0 finished with value: 0.670894526034713 and parameters: {'lr': 0.000114499798645786, 'optimizer': 'SGD', 'momentum': 0.7313573362927138}. Best is trial 0 with value: 0.670894526034713.
[I 2025-02-11 22:03:53,520] Trial 1 finished with value: 0.09078771695594126 and parameters: {'lr': 1.4170738673709922e-05, 'optimizer': 'SGD', 'momentum': 0.6587469610401484}. Best is trial 0 with value: 0.670894526034713.
[I 2025-02-11 22:07:25,593] Trial 2 finished with value: 0.7983978638184246 and parameters: {'lr': 0.00030586915274607665, 'optimizer': 'SGD', 'momen

In [28]:
print("Best trial:")
best_trial = study.best_trial
print(f"\tValue (Accuracy): {best_trial.value}")
print("\tParams: ")
for key, value in best_trial.params.items():
    print(f"\t\t{key}: {value}")


Best trial:
	Value (Accuracy): 0.9465954606141522
	Params: 
		lr: 0.007305884746406324
		optimizer: SGD
		momentum: 0.5353791587657116


In [ ]:
import matplotlib.pyplot as plt
import csv
import copy
import xlsxwriter

def train_model(model, criterion, optimizer, num_epochs=25, patience=5):

    csv_file = open('metrics_log.csv', 'w', newline='')
    writer = csv.writer(csv_file)

    # Write a header row
    writer.writerow(['Epoch', 'TrainLoss', 'ValLoss', 'TrainAcc', 'ValAcc'])

    workbook = xlsxwriter.Workbook('metrics_log_aid_dataset.xlsx')
    worksheet = workbook.add_worksheet('Metrics')
    
    # Write headers
    headers = ['Epoch', 'TrainLoss', 'ValLoss', 'TrainAcc', 'ValAcc']
    for col, header in enumerate(headers):
        worksheet.write(0, col, header)




    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    epochs_no_improve = 0
    
    # Lists to store results for plotting
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        ###################
        # Training phase
        ###################
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Accumulate training loss
            running_loss += loss.item() * images.size(0)

            # Compute training accuracy
            _, preds = torch.max(outputs, 1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)

        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = correct_train / total_train

        ###################
        # Validation phase
        ###################
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                running_val_loss += loss.item() * images.size(0)

                _, preds = torch.max(outputs, 1)
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)

        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        epoch_val_acc = correct_val / total_val

        # Print epoch results
        print(f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}")
        print(f"Val Loss:   {epoch_val_loss:.4f}, Val Acc:   {epoch_val_acc:.4f}")

        # Append for plotting
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)

        writer.writerow([epoch+1, epoch_train_loss, epoch_val_loss, epoch_train_acc, epoch_val_acc])

        # Early stopping check
        if epoch_val_acc > best_acc:
            best_acc = epoch_val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print("Early stopping triggered!")
                break

    # Load best model weights
    model.load_state_dict(best_model_wts)


    # Write data
    for row in range(len(train_losses)):
        worksheet.write(row+1, 0, row+1)  # Epoch
        worksheet.write(row+1, 1, train_losses[row])
        worksheet.write(row+1, 2, val_losses[row])
        worksheet.write(row+1, 3, train_accuracies[row])
        worksheet.write(row+1, 4, val_accuracies[row])
    
    # Create a chart for Loss
    chart_loss = workbook.add_chart({'type': 'line'})
    # Add series for TrainLoss (column 1)
    chart_loss.add_series({
        'name': 'TrainLoss',
        'categories': ['Metrics', 1, 0, len(train_losses), 0],  # epoch numbers
        'values': ['Metrics', 1, 1, len(train_losses), 1],      # train loss
    })
    # Add series for ValLoss (column 2)
    chart_loss.add_series({
        'name': 'ValLoss',
        'categories': ['Metrics', 1, 0, len(train_losses), 0],
        'values': ['Metrics', 1, 2, len(train_losses), 2],
    })
    chart_loss.set_title({'name': 'Loss over Epochs'})
    chart_loss.set_x_axis({'name': 'Epoch'})
    chart_loss.set_y_axis({'name': 'Loss'})
    
    # Insert chart into the worksheet
    worksheet.insert_chart('G2', chart_loss)
    
    # Create a chart for Accuracy
    chart_acc = workbook.add_chart({'type': 'line'})
    # Add series for TrainAcc (column 3)
    chart_acc.add_series({
        'name': 'TrainAcc',
        'categories': ['Metrics', 1, 0, len(train_losses), 0],  # epoch
        'values': ['Metrics', 1, 3, len(train_losses), 3],      # train acc
    })
    # Add series for ValAcc (column 4)
    chart_acc.add_series({
        'name': 'ValAcc',
        'categories': ['Metrics', 1, 0, len(train_losses), 0],
        'values': ['Metrics', 1, 4, len(train_losses), 4],
    })
    chart_acc.set_title({'name': 'Accuracy over Epochs'})
    chart_acc.set_x_axis({'name': 'Epoch'})
    chart_acc.set_y_axis({'name': 'Accuracy'})

    worksheet.insert_chart('G18', chart_acc)
    
    workbook.close()

    csv_file.close()

    # After training finishes, plot the results
    epochs_range = range(1, len(train_losses) + 1)

    plt.figure(figsize=(12, 5))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, train_losses, label='Train Loss')
    plt.plot(epochs_range, val_losses, label='Val Loss')
    plt.title('Loss over epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, train_accuracies, label='Train Acc')
    plt.plot(epochs_range, val_accuracies, label='Val Acc')
    plt.title('Accuracy over epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

    return model, best_acc


In [30]:
if best_trial.params["optimizer"] == "Adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=best_trial.params["lr"])
else:  # SGD
    optimizer = torch.optim.SGD(model.parameters(), lr=best_trial.params["lr"], momentum=best_trial.params["momentum"])

model, best_val_acc = train_model(model, criterion, optimizer, num_epochs=25, patience=5)

def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

test_acc = evaluate_model(model, test_loader)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/25
Validation Accuracy: 0.9239
Epoch 2/25
Validation Accuracy: 0.9466
Epoch 3/25
Validation Accuracy: 0.9513
Epoch 4/25
Validation Accuracy: 0.9479
Epoch 5/25
Validation Accuracy: 0.9519
Epoch 6/25
Validation Accuracy: 0.9479
Epoch 7/25
Validation Accuracy: 0.9553
Epoch 8/25
Validation Accuracy: 0.9593
Epoch 9/25
Validation Accuracy: 0.9573
Epoch 10/25
Validation Accuracy: 0.9606
Epoch 11/25
Validation Accuracy: 0.9626
Epoch 12/25
Validation Accuracy: 0.9666
Epoch 13/25
Validation Accuracy: 0.9586
Epoch 14/25
Validation Accuracy: 0.9640
Epoch 15/25
Validation Accuracy: 0.9559
Epoch 16/25
Validation Accuracy: 0.9619
Epoch 17/25
Validation Accuracy: 0.9720
Epoch 18/25
Validation Accuracy: 0.9593
Epoch 19/25
Validation Accuracy: 0.9626
Epoch 20/25
Validation Accuracy: 0.9626
Epoch 21/25
Validation Accuracy: 0.9619
Epoch 22/25
Validation Accuracy: 0.9539
Early stopping triggered!
Test Accuracy: 0.9640
